In [2]:
import warnings
from math import sqrt
import pandas as pd
from statsmodels.tsa.arima.model import ARIMA
from sklearn.metrics import mean_squared_error

data="data/"

In [3]:
present_data=pd.read_csv(data+"present_data.csv")

In [4]:
present_data

,Date,S&P500,Dow Jones,NASDAQ,Russel 2000,VIX Volatility,DAX Index,Shenzhen Component Index,Nikkei 225,Hang Seng Index,Crude Oil,Silver,USD-CNY,USD-EUR,USD-JPY,USD-HKD,Dollar Index,Gold
0,2003-12-01,1070.119995,9899.049805,1989.819946,554.590027,16.770000,4572.071145,404.457302,95.294217,1604.248609,29.950001,5.445000,8.267201,0.83577,109.169998,7.76500,90.430000,402.700012
1,2003-12-02,1066.619995,9853.639648,1980.069946,553.599976,16.270000,4605.004872,403.564632,95.743126,1599.637941,30.780001,5.496000,8.267201,0.82720,108.730003,7.75940,89.669998,403.700012
2,2003-12-03,1064.729980,9873.419922,1960.250000,545.190002,16.629999,4698.452979,406.144810,95.349856,1592.853458,31.100000,5.468000,8.267100,0.82488,108.300003,7.76040,89.570000,403.899994
3,2003-12-04,1069.719971,9930.820312,1968.800049,544.150024,16.299999,4681.099247,407.707746,96.333154,1589.257442,31.260000,5.452000,8.267000,0.82775,108.269997,7.76630,89.629997,403.299988
4,2003-12-05,1061.500000,9862.679688,1937.819946,539.010010,17.090000,4681.896125,405.615153,96.389702,1587.011193,30.730000,5.472000,8.266900,0.82055,107.620003,7.75970,89.190002,406.399994
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3023,2019-12-19,3205.370117,28376.960938,8887.219727,1667.089966,12.500000,14686.319862,1470.058504,217.842366,3569.611468,61.220001,17.058001,7.004000,0.89961,109.551003,7.78810,96.955002,1478.199951
3024,2019-12-20,3221.219971,28455.089844,8924.959961,1671.900024,12.510000,14818.536005,1459.396010,217.736121,3575.817105,60.439999,17.128000,7.009400,0.89880,109.383003,7.79440,97.274002,1474.699951
3025,2019-12-23,3224.010010,28551.529297,8945.650391,1674.140015,12.610000,14737.274863,1435.432548,217.546363,3579.190240,60.520000,17.403000,7.005700,0.90254,109.499001,7.79685,97.235001,1482.500000
3026,2019-12-27,3240.020020,28645.259766,9006.620117,1669.030029,13.430000,14805.686330,1462.740192,217.612612,3624.265852,61.720001,17.849001,6.996300,0.90081,109.542000,7.78790,96.545998,1513.800049


In [ ]:

 
# evaluate an ARIMA model for a given order (p,d,q)
def evaluate_arima_model(X, arima_order):
	# prepare training dataset
	train_size = int(len(X) * 0.66)
	train, test = X[0:train_size], X[train_size:]
	history = [x for x in train]
	# make predictions
	predictions = list()
	for t in range(len(test)):
		model = ARIMA(history, order=arima_order)
		model_fit = model.fit()
		yhat = model_fit.forecast()[0]
		predictions.append(yhat)
		history.append(test[t])
	# calculate out of sample error
	rmse = sqrt(mean_squared_error(test, predictions))
	return rmse
 
# evaluate combinations of p, d and q values for an ARIMA model
def evaluate_models(dataset, p_values, d_values, q_values):
	dataset = dataset.astype('float32')
	best_score, best_cfg = float("inf"), None
	for p in p_values:
		for d in d_values:
			for q in q_values:
				order = (p,d,q)
				try:
					rmse = evaluate_arima_model(dataset, order)
					print(rmse)
					if rmse < best_score:
						best_score, best_cfg = rmse, order
					print('ARIMA%s RMSE=%.3f' % (order,rmse))
				except:
					continue
	print('Best ARIMA%s RMSE=%.3f' % (best_cfg, best_score))
 

In [9]:
gold = pd.read_csv(data+'gold_dataset.csv', header=0, index_col=0, squeeze=True)["Close"]
gold=gold[0:int(len(gold)*0.1)]
gold

/tmp/ipykernel_29271/347674754.py:1: FutureWarning: The squeeze argument has been deprecated and will be removed in a future version. Append .squeeze("columns") to the call to squeeze.


  gold = pd.read_csv(data+'gold_dataset.csv', header=0, index_col=0, squeeze=True)["Close"]


Date
2000-08-30    273.899994
2000-08-31    278.299988
2000-09-01    277.000000
2000-09-05    275.799988
2000-09-06    274.200012
                 ...    
2002-08-06    305.500000
2002-08-07    314.299988
2002-08-08    310.399994
2002-08-09    314.200012
2002-08-12    314.100006
Name: Close, Length: 484, dtype: float64

In [11]:
# evaluate parameters
p_values = [0, 1, 2, 4, 6, 8, 10]
d_values = range(0, 3)
q_values = range(0, 3)
warnings.filterwarnings("ignore")
evaluate_models(gold.values, p_values, d_values, q_values)

ARIMA(0, 0, 0) RMSE=29.409
ARIMA(0, 0, 2) RMSE=10.620
ARIMA(0, 1, 0) RMSE=2.726
ARIMA(0, 1, 1) RMSE=2.726
ARIMA(0, 1, 2) RMSE=2.732
ARIMA(0, 2, 0) RMSE=3.949
ARIMA(0, 2, 1) RMSE=2.728
ARIMA(0, 2, 2) RMSE=2.728
ARIMA(1, 0, 0) RMSE=2.792
ARIMA(1, 0, 1) RMSE=2.788
ARIMA(1, 0, 2) RMSE=2.801
ARIMA(1, 1, 0) RMSE=2.727
ARIMA(1, 1, 1) RMSE=2.730
ARIMA(1, 1, 2) RMSE=2.727
ARIMA(1, 2, 0) RMSE=3.345
ARIMA(1, 2, 1) RMSE=2.728
ARIMA(1, 2, 2) RMSE=2.728
ARIMA(2, 0, 0) RMSE=2.787
ARIMA(2, 0, 1) RMSE=2.805
ARIMA(2, 0, 2) RMSE=2.797
ARIMA(2, 1, 0) RMSE=2.733
ARIMA(2, 1, 1) RMSE=2.729
ARIMA(2, 1, 2) RMSE=2.729
ARIMA(2, 2, 0) RMSE=3.012
ARIMA(2, 2, 1) RMSE=2.735
ARIMA(2, 2, 2) RMSE=2.731
ARIMA(4, 0, 0) RMSE=2.786
ARIMA(4, 0, 1) RMSE=2.791
ARIMA(4, 0, 2) RMSE=2.781
ARIMA(4, 1, 0) RMSE=2.735
ARIMA(4, 1, 1) RMSE=2.701
ARIMA(4, 1, 2) RMSE=2.703
ARIMA(4, 2, 0) RMSE=2.954
ARIMA(4, 2, 1) RMSE=2.735
ARIMA(4, 2, 2) RMSE=2.718
ARIMA(6, 0, 0) RMSE=2.785
